# AWS Bedrock Access & Quota Checker (November 2024)

Comprehensive AWS Bedrock setup validation and troubleshooting tool, focusing on Claude models access.

## ⚠️ Security Warning
This notebook will output sensitive information. Before committing:
1. Clear all cell outputs containing:
   - AWS Account IDs
   - IAM User/Role names
   - Access Key IDs
   - ARNs
   - Account-specific quotas
2. Run clean_notebooks.py
3. Verify no sensitive data remains

## Model Types
1. Direct Model Access
   - Regular model IDs (e.g., `anthropic.claude-3-sonnet-20240229-v1:0`)
   - Region-specific access
   
2. Cross-Region Inference
   - US profiles (e.g., `us.anthropic.claude-3-sonnet-20240229-v1:0`)
   - EU profiles (e.g., `eu.anthropic.claude-3-sonnet-20240229-v1:0`)
   - Multi-region access

## Prerequisites
1. AWS credentials configured (via assume/granted recommended)
2. Required packages:
```bash
pip install -r requirements.txt
```

In [ ]:
import boto3
import json
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from tabulate import tabulate
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import time

@dataclass
class BedrockModel:
    name: str
    model_id: str
    cross_region_us: Optional[str] = None
    cross_region_eu: Optional[str] = None

# Latest Claude Models (November 2024)
CLAUDE_MODELS = {
    "claude-3-haiku": BedrockModel(
        name="Claude 3 Haiku",
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        cross_region_us="us.anthropic.claude-3-haiku-20240307-v1:0",
        cross_region_eu="eu.anthropic.claude-3-haiku-20240307-v1:0"
    ),
    "claude-3-sonnet": BedrockModel(
        name="Claude 3 Sonnet",
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        cross_region_us="us.anthropic.claude-3-sonnet-20240229-v1:0",
        cross_region_eu="eu.anthropic.claude-3-sonnet-20240229-v1:0"
    ),
    "claude-3-opus": BedrockModel(
        name="Claude 3 Opus",
        model_id="anthropic.claude-3-opus-20240229-v1:0",
        cross_region_us="us.anthropic.claude-3-opus-20240229-v1:0"
    ),
    "claude-3.5-sonnet": BedrockModel(
        name="Claude 3.5 Sonnet",
        model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
        cross_region_us="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
        cross_region_eu="eu.anthropic.claude-3-5-sonnet-20241022-v2:0"
    )
}

# Region Configurations
US_REGIONS = {
    "us-east-1": "N. Virginia",
    "us-east-2": "Ohio",
    "us-west-2": "Oregon"
}

EU_REGIONS = {
    "eu-central-1": "Frankfurt",
    "eu-west-1": "Ireland",
    "eu-west-3": "Paris"
}

ASIA_REGIONS = {
    "ap-northeast-1": "Tokyo",
    "ap-southeast-1": "Singapore",
    "ap-southeast-2": "Sydney",
    "ap-south-1": "Mumbai"
}

ALL_REGIONS = {**US_REGIONS, **EU_REGIONS, **ASIA_REGIONS}

print("📋 Configuration loaded...")
print(f"Models available: {len(CLAUDE_MODELS)}")
print(f"Regions to check: {len(ALL_REGIONS)}")

## 1. AWS Identity and Permissions Check

First, verify AWS credentials and required permissions for Bedrock access.
⚠️ NOTE: This section will output sensitive account information.

In [ ]:
def check_aws_identity() -> Dict[str, Any]:
    """Verify AWS identity and permissions
    Returns dict with identity info and permissions status
    """
    results = {}
    
    # 1. Check Identity
    print("🔍 Checking AWS Identity...")
    try:
        sts = boto3.client('sts')
        identity = sts.get_caller_identity()
        results['identity'] = {
            'account_id': identity['Account'],
            'arn': identity['Arn'],
            'user_id': identity['UserId']
        }
        print("✓ Identity confirmed")
        print(f"Account: {results['identity']['account_id']}")
        print(f"User ID: {results['identity']['user_id']}")
    except Exception as e:
        print(f"❌ Identity check failed: {e}")
        return results
    
    # 2. Check Bedrock Access
    print("\n🔍 Checking Bedrock Access...")
    try:
        bedrock = boto3.client('bedrock', region_name='us-east-1')
        models = bedrock.list_foundation_models()
        claude_models = [m for m in models['modelSummaries'] 
                        if m.get('providerName') == 'Anthropic']
        print(f"✓ Found {len(claude_models)} Claude models")
        results['bedrock_access'] = True
    except Exception as e:
        print(f"❌ Bedrock access check failed: {e}")
        results['bedrock_access'] = False

    # 3. Check Admin Access
    print("\n🔍 Checking IAM Access...")
    try:
        iam = boto3.client('iam')
        response = iam.get_user()
        if 'User' in response:
            print("✓ IAM access confirmed")
            results['iam_access'] = True
            
            # List policies without using username
            caller_arn = identity['Arn']
            if 'AdministratorAccess' in str(response):
                print("✓ Administrator access detected")
            else:
                print("\nAccess Level Check:")
                print("- Bedrock access: Available (found Claude models)")
                print("- IAM access: Limited (not admin)")
                print("\nRequired Bedrock Permissions:")
                print("- bedrock:InvokeModel")
                print("- bedrock:ListFoundationModels")
                print("- bedrock:GetFoundationModel")
                print("\nRecommendation:")
                print("If you experience permission issues, ask your AWS admin to:")
                print("1. Grant BedrockFullAccess policy")
                print("2. Or create custom policy with above permissions")
                print("3. Enable model access in Bedrock console")
    except Exception as e:
        print(f"❌ IAM access check failed: {e}")
        results['iam_access'] = False
    
    print("\n⚠️ SECURITY NOTE: Clear this output before committing!")
    return results

# Run identity check
identity_results = check_aws_identity()

## 2. Bedrock Service Check

Verify Bedrock service access and list available models across regions.

In [ ]:
def check_bedrock_access(region: str) -> Dict[str, Any]:
    """Check Bedrock access in specific region
    Returns dict with access status and available models
    """
    results = {
        'region': region,
        'region_name': ALL_REGIONS.get(region, 'Unknown'),
        'status': 'Unknown',
        'models': [],
        'claude_models': [],
        'inference_profiles': [],
        'errors': []
    }
    
    try:
        bedrock = boto3.client('bedrock', region_name=region)
        
        # List models
        models = bedrock.list_foundation_models()
        for model in models['modelSummaries']:
            model_info = {
                'id': model['modelId'],
                'provider': model.get('providerName', 'Unknown'),
                'status': model.get('modelLifecycle', {}).get('status', 'Unknown'),
                'types': model.get('inferenceTypesSupported', [])
            }
            results['models'].append(model_info)
            
            # Track Claude models separately
            if model.get('providerName') == 'Anthropic':
                results['claude_models'].append(model_info)
        
        # Check inference profiles
        try:
            profiles = bedrock.list_inference_profiles()
            results['inference_profiles'] = profiles.get('inferenceProfiles', [])
        except Exception as e:
            results['errors'].append(f"Inference profiles error: {e}")
        
        results['status'] = 'Available'
        
    except Exception as e:
        results['status'] = 'Error'
        results['errors'].append(str(e))
    
    return results

# Check first US region as example
first_region = next(iter(US_REGIONS))
print(f"Checking Bedrock access in {first_region}...")
region_check = check_bedrock_access(first_region)

# Print Claude models found
print(f"\nFound {len(region_check['claude_models'])} Claude models:")
for model in region_check['claude_models']:
    print(f"- {model['id']} ({', '.join(model['types'])})")

## 3. Cross-Region Inference Profile Check

Test availability of cross-region inference profiles and their supported regions.
This is particularly important for Claude 3.5 models.

In [ ]:
def check_inference_profiles(region_group: str = 'US') -> Dict[str, Any]:
    """Check cross-region inference profiles
    Args:
        region_group: 'US' or 'EU'
    """
    results = {
        'group': region_group,
        'profiles': [],
        'errors': []
    }
    
    # Select primary region for checking
    primary_region = 'us-east-1' if region_group == 'US' else 'eu-central-1'
    prefix = f"{region_group.lower()}.anthropic"
    
    try:
        bedrock = boto3.client('bedrock', region_name=primary_region)
        profiles = bedrock.list_inference_profiles()
        
        # Filter for relevant profiles
        for profile in profiles.get('inferenceProfiles', []):
            if profile['inferenceProfileId'].startswith(prefix):
                profile_info = {
                    'id': profile['inferenceProfileId'],
                    'regions': profile.get('awsRegions', []),
                    'arn': profile.get('inferenceProfileArn')
                }
                results['profiles'].append(profile_info)
    
    except Exception as e:
        results['errors'].append(f"Profile check error: {str(e)}")
    
    return results

# Check both US and EU profiles
print("Checking Cross-Region Inference Profiles...\n")

us_profiles = check_inference_profiles('US')
eu_profiles = check_inference_profiles('EU')

# Print findings
for group in [us_profiles, eu_profiles]:
    print(f"\n{group['group']} Profiles:")
    if group['profiles']:
        for profile in group['profiles']:
            print(f"\nProfile: {profile['id']}")
            print(f"Regions: {', '.join(profile['regions'])}")
    else:
        print("No profiles found")
    
    if group['errors']:
        print("\nErrors:")
        for error in group['errors']:
            print(f"- {error}")

## 4. Quota and Limit Checks

Check service quotas and usage limits across regions.
⚠️ NOTE: Quota values are account-specific and should be cleared before commit.

In [ ]:
def check_bedrock_quotas(region: str) -> Dict[str, Any]:
    """Check Bedrock quotas in a specific region
    Returns quota information for Claude models
    """
    results = {
        'region': region,
        'quotas': [],
        'errors': []
    }
    
    try:
        quotas = boto3.client('service-quotas', region_name=region)
        response = quotas.list_service_quotas(ServiceCode='bedrock')
        
        # Filter for relevant quotas
        for quota in response['Quotas']:
            if any(term in quota['QuotaName'].lower() for term in 
                  ['invoke', 'token', 'anthropic', 'claude']):
                quota_info = {
                    'name': quota['QuotaName'],
                    'value': quota['Value'],
                    'adjustable': quota['Adjustable'],
                    'unit': quota.get('Unit', 'Count')
                }
                results['quotas'].append(quota_info)
                
    except Exception as e:
        results['errors'].append(f"Quota check error: {str(e)}")
    
    return results

def print_quota_table(quotas: List[Dict[str, Any]], region: str):
    """Print quota information in a readable table"""
    if not quotas:
        print(f"No quotas found in {region}")
        return
    
    headers = ['Quota Name', 'Value', 'Adjustable']
    rows = [[q['name'], q['value'], '✓' if q['adjustable'] else '✗'] 
            for q in quotas]
    
    print(f"\nQuotas in {region}:")
    print(tabulate(rows, headers=headers, tablefmt='grid'))

# Check quotas in main regions
print("Checking quotas across regions...")
for region in ['us-east-1', 'us-west-2', 'ap-northeast-1']:
    quota_results = check_bedrock_quotas(region)
    print_quota_table(quota_results['quotas'], region)
    
    if quota_results['errors']:
        print(f"\nErrors in {region}:")
        for error in quota_results['errors']:
            print(f"- {error}")

print("\n⚠️ SECURITY NOTE: Clear quota values before committing!")

## 5. Model Testing

Test actual model invocation with different access methods:
1. Direct model access
2. Cross-region inference profiles
3. Different regions

In [ ]:
def test_model_invocation(model_id: str, 
                         region: str = 'us-east-1',
                         max_retries: int = 3,
                         delay: int = 30,
                         is_inference_profile: bool = False) -> Dict[str, Any]:
    """Test model invocation with retry logic"""
    from anthropic import AnthropicBedrock
    
    print(f"Testing {model_id} in {region}...")
    results = {
        'model_id': model_id,
        'region': region,
        'success': False,
        'response': None,
        'error': None,
        'attempts': 0
    }
    
    client = AnthropicBedrock(aws_region=region)
    
    for attempt in range(max_retries):
        try:
            results['attempts'] += 1
            response = client.messages.create(
                model=model_id,
                max_tokens=256,
                messages=[
                    {
                        "role": "user",
                        "content": "Say hello and confirm your model name!"
                    }
                ]
            )
            results['success'] = True
            results['response'] = response.content[0].text
            return results
            
        except Exception as e:
            error_msg = str(e)
            print(f"Attempt {attempt + 1} failed: {error_msg}")
            results['error'] = error_msg
            
            if attempt < max_retries - 1:
                print(f"Waiting {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
    
    return results

# Test configuration
test_scenarios = [
    # Direct model access
    {'model': CLAUDE_MODELS['claude-3-sonnet'].model_id, 'region': 'us-east-1'},
    
    # Cross-region inference
    {'model': CLAUDE_MODELS['claude-3.5-sonnet'].cross_region_us, 'region': 'us-east-1'},
    {'model': CLAUDE_MODELS['claude-3.5-sonnet'].cross_region_us, 'region': 'us-west-2'},
    
    # Different regions
    {'model': CLAUDE_MODELS['claude-3-haiku'].model_id, 'region': 'ap-northeast-1'}
]

print("Running model tests...\n")
test_results = []

for scenario in test_scenarios:
    result = test_model_invocation(**scenario)
    test_results.append(result)
    print(f"\nResult for {scenario['model']} in {scenario['region']}:")
    print(f"Success: {'✓' if result['success'] else '✗'}")
    if result['success']:
        print(f"Response: {result['response']}")
    else:
        print(f"Error: {result['error']}")
    print("-" * 50)

## Troubleshooting Guide

Common issues and solutions:

1. Rate Limits (429 errors):
   - Check quotas in current region
   - Try different regions (especially ap-northeast-1)
   - Use cross-region inference profiles
   - Implement longer delays between requests

2. Access Denied (403 errors):
   - Verify model access is enabled in console
   - Check IAM permissions
   - Ensure AWS credentials are correct
   - Try different regions

3. Invalid Model ID:
   - Verify model availability in region
   - Check if cross-region inference is required
   - Confirm model ID format matches region type

4. Cross-Region Issues:
   - Use correct profile format (us. or eu. prefix)
   - Verify profile availability in region
   - Check supported regions for profile

## References
- [Claude on Amazon Bedrock](https://docs.anthropic.com/claude/docs/claude-on-amazon-bedrock)
- [AWS Bedrock Quotas](https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html)
- [Cross-Region Inference](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-support.html)

## Security Reminders
1. Clear all notebook outputs before committing
2. Run clean_notebooks.py
3. Verify no sensitive data in stored files